In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import matplotlib as mpl
from jupyterthemes import jtplot #These two lines can be skipped if you are not using jupyter themes
jtplot.reset()

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.314)
import scipy as sp
import multiprocessing as mp


import time
start_total = time.time()

In [2]:
import os
my_path = '/home/tomi/Documentos/Fisica/Tesis/escrito-tesis/images/'

In [3]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

In [4]:
zl = 0.2; zs = 1.2
Dl = cosmo.angular_diameter_distance(zl)   
Ds = cosmo.angular_diameter_distance(zs)    
Dls = cosmo.angular_diameter_distance_z1z2(zl, zs)
G = const.G
rho_crit = (cosmo.critical_density(zl)).to(u.kg/u.m**3)
c_light = (const.c).to(u.cm/u.second)

#r0 = 10*u.kpc
r0 = 10.0*u.kpc
#r0 = 0.1*u.kpc
pi = np.pi

def scale_radius(v,Dl,Ds,Dls):                               #this is e0 in eq 3.42 meneghetti, eq 1 barnacka 2014
    return (4.*pi*v**2/c_light**2*Dl*Dls/Ds).decompose()
def theta_E_SIS():
    'in arcsec'
    pre_theta_E = (scale_radius(v,Dl,Ds,Dls)/Dl).decompose()
    return pre_theta_E*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 

v = 180 *u.km/u.s
ss_r = scale_radius(v,Dl,Ds,Dls) 
print('scale radius (m): ',ss_r)
print('scale radius (kpc): ',ss_r.to(u.kpc))
print('theta_E: ',theta_E_SIS() ,'arcsec')
theta_E_num = theta_E_SIS()
elipt = 0.3
re = (const.e.esu**2/const.m_e/(c_light**2)).decompose()
print('Classic electron radius: ',re)

scale radius (m):  7.705329461274929e+19 m
scale radius (kpc):  2.49712721364453 kpc
theta_E:  0.7301786792241515 arcsec
Classic electron radius:  2.817940324670788e-15 m


The lensing potential by a point mass is given by 

$$
\psi = \frac{4GM}{c^2} \frac{D_{ls}}{D_l D_s} ln |\vec{\theta}|
$$

In terms of the Einstein radius

$$
\theta_{e_1} = \sqrt{\frac{4GM}{c^2} \frac{D_{ls}}{D_l D_s}} \\
\psi = \theta_{e_1}^2 ln |\vec{\theta}|
$$

Lets start with 

$$
M_1 = 10^3 M_\odot \\
M_2 = 10^4 M_\odot \\
M_3 = 10^5 M_\odot \\
M_4 = 10^6 M_\odot \\
M_5 = 10^8 M_\odot 
$$

The mass of the main lens is 

$$
M(\theta_e) =  \theta_e^2 \frac{c^2}{4G} \frac{D_l D_s}{D_{ls}}
$$

In [5]:
M_e = (theta_E_num*u.arcsec)**2 * c_light**2 /4 / G * Dl * Ds / Dls
M_e = (M_e/(u.rad)**2).decompose()
ms = 1.98847e30*u.kg
M_e/ms

<Quantity 5.90964413e+10>

$$
M(0.73 \mathrm{arcsec}) =  5.91\cdot 10^{10} M_\odot
$$

In [6]:
ms = 1.98847e30*u.kg  #solar mass
m1 = ms*1e3
m2 = ms*1e4
m3 = ms*1e5
m4 = ms*1e6
m5 = ms*1e8

In [7]:
theta_E_1 = np.sqrt(4*G*m1/c_light**2*Dls/Dl/Ds)
theta_E_1 = theta_E_1.decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
theta_E_1.value

9.498356879908454e-05

In [8]:
theta_E_2 = np.sqrt(4*G*m2/c_light**2*Dls/Dl/Ds)
theta_E_2 = theta_E_2.decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
theta_E_2.value

0.0003003644176964114

In [9]:
theta_E_3 = np.sqrt(4*G*m3/c_light**2*Dls/Dl/Ds)
theta_E_3 = theta_E_3.decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
theta_E_3.value

0.0009498356879908454

In [10]:
theta_E_4 = np.sqrt(4*G*m4/c_light**2*Dls/Dl/Ds)
theta_E_4 = theta_E_4.decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
theta_E_4.value

0.0030036441769641137

In [11]:
theta_E_5 = np.sqrt(4*G*m5/c_light**2*Dls/Dl/Ds)
theta_E_5 = theta_E_5.decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
theta_E_5.value

0.030036441769641136

### Two black holes of $ M_4 = 10^6 M_\odot$

In [44]:
x1 = -0.26631755 + 2e-3
y1 = -0.26631755 

x0 = y0 = - 0.26631755
b1x = -3.5e-3 + x0 ; b1y = -2e-3 + y0
b2x = 0 + x0 ; b2y = -1.5e-3 + y0
b3x = 3.5e-3 + x0 ; b3y = 2.5e-3 + y0

In [45]:
x2 = -0.26631755 - 2e-3
y2 = -0.26631755 - 4e-3

In [46]:
lens_model_list = ['SIEBH2']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)

In [47]:
kwargs = {'theta_E':theta_E_num.value,'eta':0*elipt, 'theta_E_1':theta_E_4.value, 'x1':x1, 'y1':y1, 'theta_E_2':theta_E_4.value, 'x2':x2, 'y2':y2}
kwargs_lens_list = [kwargs]

In [55]:
from lenstronomy.LensModel.Profiles.sie_black_hole_2 import SIEBH2
perfil = SIEBH2()

t = [0,0,0,0]
phi = [0,0,0,0]

phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2])*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3])*(u.arcsec**2).to('rad**2')).to('s').value



print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

-8.105748056903394
-3.967594036250375
-4.138154020653019
-21792.710728007325
-21796.848882027978
-21800.81647606423


In [49]:
phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1]*0)*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2]*0)*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3]*0)*(u.arcsec**2).to('rad**2')).to('s').value


print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

-15557.484019854921
-10126.815301985247
-5430.668717869674


In [50]:
phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2])*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3])*(u.arcsec**2).to('rad**2')).to('s').value


print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

15549.378271797788
10122.84770794888
5426.530563848908


# massless black holes

In [51]:
from lenstronomy.LensModel.Profiles.sie_black_hole_2 import SIEBH2
perfil = SIEBH2()

t = [0,0,0,0]
phi = [0,0,0,0]

phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2])*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3])*(u.arcsec**2).to('rad**2')).to('s').value



print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

4.394162934288033
22.793922406694037
-18.399759472406004


In [52]:
phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1]*0)*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2]*0)*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3]*0)*(u.arcsec**2).to('rad**2')).to('s').value


print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

-15557.484019854921
-10126.815301985247
-5430.668717869674


In [53]:
phi[1] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value

phi[2] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[2] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[2])*(u.arcsec**2).to('rad**2')).to('s').value

phi[3] = SIEBH2.function(perfil, b3x, b3y, theta_E_num.value, 0*elipt, 0*theta_E_4.value, x1, y1, 0*theta_E_4.value, x2, y2)
t[3] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b3x )**2 + (.25 - b3y)**2) - phi[3])*(u.arcsec**2).to('rad**2')).to('s').value


print(t[3] - t[1])
print(t[3] - t[2])
print(t[2] - t[1])

15561.878182789194
10149.609224391985
5412.26895839721


### $ M_1 = 10^3 M_\odot$

In [56]:
x1 = -0.06630 - .05e-3
y1 = -0.0662868 
x2 = -0.06630 + .05e-3
y2 = -0.0662868 

In [57]:
lens_model_list = ['SIEBH2']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)

In [58]:
kwargs = {'theta_E':theta_E_num.value,'eta':0*elipt, 'theta_E_1':theta_E_1.value, 'x1':x1, 'y1':y1, 'theta_E_2':theta_E_1.value, 'x2':x2, 'y2':y2}
kwargs_lens_list = [kwargs]

In [59]:
x0 = -.0662968
y0 = -.0663026
#blobs position
b1x = x0 - -.1e-3 ; b1y = y0 - -.1e-3
b2x = x0 - .1e-3 ; b2y = y0 - .1e-3

In [60]:
from lenstronomy.LensModel.Profiles.sie_black_hole_2 import SIEBH2
perfil = SIEBH2()

t_ = [0,0]
phi = [0,0]

phi[0] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[0] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[0])*(u.arcsec**2).to('rad**2')).to('s').value

phi[1] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value


print(t_[0] - t_[1])

216.6083137965179


In [42]:
t_ = [0,0]
phi = [0,0]

phi[0] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[0] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[0]*0)*(u.arcsec**2).to('rad**2')).to('s').value

phi[1] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[1]*0)*(u.arcsec**2).to('rad**2')).to('s').value


print(t_[0] - t_[1])

-331.5627250271791


In [43]:
t_ = [0,0]
phi = [0,0]

phi[0] = SIEBH2.function(perfil, b1x, b1y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[0] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b1x )**2 + (.25 - b1y)**2) - phi[0])*(u.arcsec**2).to('rad**2')).to('s').value

phi[1] = SIEBH2.function(perfil, b2x, b2y, theta_E_num.value, 0*elipt, theta_E_4.value, x1, y1, theta_E_4.value, x2, y2)
t_[1] = ((1+zl)/c_light*Ds*Dl/Dls*( 0*1/2*( (.25 - b2x )**2 + (.25 - b2y)**2) - phi[1])*(u.arcsec**2).to('rad**2')).to('s').value


print(t_[0] - t_[1])

548.1710388237261


In [ ]:
end_total = time.time()
print('total time: ',(end_total-start_total)/60.,' minutes')